<a href="https://colab.research.google.com/github/MatteoD2126/INF554/blob/main/InfoPIA2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!unzip drive/MyDrive/KaggleChallenge/inf554-extractive-summarization-2023.zip

Archive:  drive/MyDrive/KaggleChallenge/inf554-extractive-summarization-2023.zip
  inflating: baseline.py             
  inflating: make_submission.py      
  inflating: submission_naive_baseline.csv  
  inflating: test/ES2003a.json       
  inflating: test/ES2003a.txt        
  inflating: test/ES2003b.json       
  inflating: test/ES2003b.txt        
  inflating: test/ES2003c.json       
  inflating: test/ES2003c.txt        
  inflating: test/ES2003d.json       
  inflating: test/ES2003d.txt        
  inflating: test/ES2004a.json       
  inflating: test/ES2004a.txt        
  inflating: test/ES2004b.json       
  inflating: test/ES2004b.txt        
  inflating: test/ES2004c.json       
  inflating: test/ES2004c.txt        
  inflating: test/ES2004d.json       
  inflating: test/ES2004d.txt        
  inflating: test/ES2011a.json       
  inflating: test/ES2011a.txt        
  inflating: test/ES2011b.json       
  inflating: test/ES2011b.txt        
  inflating: test/ES2011c.json       


In [ ]:
!pip install jsonargparse sentence-transformers
import torch
print(torch.__version__)
!pip install torch-scatter torch-sparse torch-geometric -f https://data.pyg.org/whl/torch-{torch.__version__}.html

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.7/189.7 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 12.5 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=008b0368648c3073934a6d811c856365afeb0000cad49638c1f617ffd0634aa6
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers
2.1.0+cu118
Looking in links: https://data.pyg.org/whl/torch-2.1.0+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 44.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 73.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 5.1 MB/s eta 0:00:00


In [ ]:
import json
from pathlib import Path
import numpy as np
import pandas as pd
import os
import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
import torch_geometric
from torch.nn import Parameter
from torch_geometric.utils.convert import to_networkx,from_networkx
import networkx as nx
import urllib.request
import tarfile
from torch_geometric.nn import GraphSAGE, SAGEConv, GATConv
from torch_geometric.loader import NeighborLoader
from torch_geometric.utils import to_networkx
%matplotlib inline
import torch
import sklearn.metrics
from transformers import AutoTokenizer
from torch.optim import AdamW
from torch import nn
from torch.utils.data import DataLoader,TensorDataset

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [ ]:


def flatten(list_of_list):
    return [item for sublist in list_of_list for item in sublist]

path_to_training = Path("training")
path_to_test = Path("test")

#####
# training and test sets of transcription ids
#####
training_set = ['ES2002', 'ES2005', 'ES2006', 'ES2007', 'ES2008', 'ES2009', 'ES2010', 'ES2012', 'ES2013', 'ES2015', 'ES2016', 'IS1000', 'IS1001', 'IS1002', 'IS1003', 'IS1004', 'IS1005', 'IS1006', 'IS1007', 'TS3005', 'TS3008', 'TS3009', 'TS3010', 'TS3011', 'TS3012']
training_set = flatten([[m_id+s_id for s_id in 'abcd'] for m_id in training_set])
training_set.remove('IS1002a')
training_set.remove('IS1005d')
training_set.remove('TS3012c')

test_set = ['ES2003', 'ES2004', 'ES2011', 'ES2014', 'IS1008', 'IS1009', 'TS3003', 'TS3004', 'TS3006', 'TS3007']
test_set = flatten([[m_id+s_id for s_id in 'abcd'] for m_id in test_set])

#####

In [ ]:
def encode_all(tokenizer, texts, max_length):
    token_ids = np.zeros(shape=(len(texts), max_length),
                         dtype=np.int32)
    for i, txt in enumerate(texts):
        encoded = tokenizer.encode(txt, max_length=max_length,truncation=True)
        token_ids[i, 0:len(encoded)] = encoded
    attention_mask = (token_ids != 0).astype(np.int32)
    return {"input_ids": token_ids, "attention_mask": attention_mask}


In [ ]:

y_training = []
with open("training_labels.json", "r") as file:
    training_labels = json.load(file)
X_training = []
for transcription_id in training_set:
    with open(path_to_training / f"{transcription_id}.json", "r") as file:
        transcription = json.load(file)

    for utterance in transcription:
        X_training.append(utterance["speaker"] + ": " + utterance["text"])

    y_training += training_labels[transcription_id]

In [ ]:

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
train_tokens=encode_all(tokenizer,X_training,256)


tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [ ]:

trainset = TensorDataset(torch.from_numpy(np.array([(train_tokens['input_ids']),train_tokens['attention_mask']]).transpose(1,0,2)).to(device),
                                          torch.from_numpy(np.array(y_training)).to(device))


In [ ]:

num_train_samples=int(len(trainset)*0.8)
num_val_samples=len(trainset)-num_train_samples
train_dataset,eval_dataset=torch.utils.data.random_split(trainset,[num_train_samples,num_val_samples])

train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=10)
eval_dataloader = DataLoader(eval_dataset, batch_size=10)

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("drive/MyDrive/KaggleChallenge/orion", num_labels=2)

In [ ]:

optimizer = AdamW(model.parameters(), lr=2e-5)
from transformers import get_scheduler

num_training_steps = 3 * len(train_dataloader)

lr_scheduler = get_scheduler(name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:


def eval_mod(max_iter=-1):
  all_predictions=[]
  all_references=[]
  cpt=0
  model.eval()
  for batch in eval_dataloader:
      input_ids=batch[0][:,0,:]
      attention_masks=batch[0][:,1,:]
      with torch.no_grad():
        outputs = model(input_ids=input_ids,attention_mask=attention_masks,labels=batch[1])
      logits = outputs.logits
      predictions = torch.argmax(logits, dim=-1)
      all_predictions.append(np.array(predictions.cpu()))
      all_references.append(np.array(batch[1].cpu()))
      if cpt>max_iter and max_iter>=0:
        break
      cpt+=1

  all_predictions=np.hstack(all_predictions)
  all_references=np.hstack(all_references)

  print(f'F1={sklearn.metrics.f1_score(all_references,all_predictions)}')
  return all_references,all_predictions

In [ ]:
from tqdm.auto import tqdm
num_epochs=2
num_training_steps = num_epochs * len(train_dataloader)

for epoch in range(0,num_epochs):
  model.train()
  print(f'Epoch {epoch}')
  progress_bar = tqdm(range(len(train_dataloader)))
  for batch in train_dataloader:
      input_ids=batch[0][:,0,:]
      attention_masks=batch[0][:,1,:]
      outputs = model(input_ids=input_ids,attention_mask=attention_masks,labels=batch[1])
      loss= nn.CrossEntropyLoss(weight=torch.tensor([1/0.8,1/0.2]).to(device))(outputs.logits, batch[1])
      loss.backward()
      optimizer.step()
      lr_scheduler.step()
      optimizer.zero_grad()
      progress_bar.update(1)
  eval_mod()

In [ ]:
model.save_pretrained('drive/MyDrive/KaggleChallenge/orion')

In [ ]:
# Define the original training set
from sklearn.model_selection import train_test_split
original_training_set = ['ES2002', 'ES2005', 'ES2006', 'ES2007', 'ES2008', 'ES2009', 'ES2010', 'ES2012', 'ES2013', 'ES2015', 'ES2016',
                         'IS1000', 'IS1001', 'IS1002', 'IS1003', 'IS1004', 'IS1005', 'IS1006', 'IS1007', 'TS3005', 'TS3008', 'TS3009', 'TS3010', 'TS3011', 'TS3012']
original_training_set = flatten([[m_id+s_id for s_id in 'abcd'] for m_id in original_training_set])
original_training_set.remove('IS1002a')
original_training_set.remove('IS1005d')
original_training_set.remove('TS3012c')

# Create a new split of training_set for training and testing
training_set, test_set = train_test_split(original_training_set, test_size=0.15, random_state=42)

print(len(training_set), len(test_set))

def encode_all(tokenizer, texts, max_length):
    token_ids = np.zeros(shape=(len(texts), max_length),
                         dtype=np.int32)
    for i, txt in enumerate(texts):
        encoded = tokenizer.encode(txt, max_length=max_length,truncation=True)
        token_ids[i, 0:len(encoded)] = encoded
    attention_mask = (token_ids != 0).astype(np.int32)
    return {"input_ids": token_ids, "attention_mask": attention_mask}

y_training = []
with open("training_labels.json", "r") as file:
    training_labels = json.load(file)
X_training = []
for transcription_id in training_set:
    with open(path_to_training / f"{transcription_id}.json", "r") as file:
        transcription = json.load(file)

    for utterance in transcription:
        X_training.append(utterance["speaker"] + ": " + utterance["text"])

    y_training += training_labels[transcription_id]

82 15


In [ ]:

def create_dir_graph(transcription_id,path=path_to_training):

    G = nx.DiGraph()
    #load the json for the texts and infos to creates nodes
    node_infos = []

    with open(path / f"{transcription_id}.json", "r") as file:
        transcription = json.load(file)

    for utterance in transcription:
        node_infos.append([utterance["index"], utterance["speaker"], utterance["text"]])

    # Add nodes and edges based on speaker information

    for utterance_3 in node_infos:
        G.add_node(utterance_3[0], speaker=utterance_3[1], text=utterance_3[2])

    #load the txt for direction and type of interaction
    #print(path_to_training / f"{transcription_id}.txt")
    with open(path / f"{transcription_id}.txt", "r") as txt_file:
        discourse_data = txt_file.readlines()

    for line in discourse_data:
        tokens = line.split()
        if len(tokens) == 3:
            source_node = int(tokens[0])
            target_node = int(tokens[2])
            edge_type = tokens[1]
            #print(line, "DETAIL:","source", source_node, "target", target_node, "edge", edge_type)
            G.add_edge(source_node, target_node, type=edge_type)
            #G.add_edge(target_node, source_node, type=edge_type)
    return G

# Load or create labels for the original training set
with open("training_labels.json", "r") as file:
    original_training_labels = json.load(file)

print("numbers of keys in dict original training labels: ", len(original_training_labels.keys()))

# Create labels for the new training set
training_labels = {transcription_id: original_training_labels[transcription_id] for transcription_id in training_set}
print("numbers of keys in dict training labels: ", len(training_labels.keys()))

y_training = []
X_training = []

for transcription_id in training_set:
    # Add the graph for this conversation
    X_training.append(create_dir_graph(transcription_id))
    y_training.append(original_training_labels[transcription_id])

y_test = []
X_test = []

for transcription_id in test_set:
    # Add the graph for this conversation
    X_test.append(create_dir_graph(transcription_id))
    y_test.append(original_training_labels[transcription_id])
print("len of test set ", len(X_training))

numbers of keys in dict original training labels:  97
numbers of keys in dict training labels:  82
len of test set  82


In [ ]:
len(X_test),len(y_test)

(15, 15)

In [ ]:
def create_data(X_training,y_training):
  fnodes=[]
  edges=[]
  flabels=[]
  number=0
  for G,labels in zip(X_training,y_training):
    for ind in G.nodes:
      node=G.nodes[ind]
      text=node['speaker']+': '+node['text']
      fnodes.append(text)
      flabels.append(labels[ind])
    for edge1,edge2 in G.edges:
      #edges.append([edge1+number,edge2+number])
      edges.append([edge2+number,edge1+number])
    number+=ind
  encoded_nodes=encode_all(tokenizer,fnodes,256)

  flabels=np.array(flabels)
  ran_nodes=np.zeros((flabels.shape[0],768))
  cpt=0
  for k in range(0,len(encoded_nodes['input_ids']),10):
      input_id=encoded_nodes['input_ids'][k:(k+10)]
      mask=encoded_nodes['attention_mask'][k:(k+10)]
      with torch.no_grad():
        out = model.bert(input_ids=torch.from_numpy(input_id).to(device),attention_mask=torch.from_numpy(mask).to(device))
        ran_nodes[k:(k+10),:]=out.pooler_output.cpu()

  edges=np.array(edges)
  data=torch_geometric.data.Data(x=torch.from_numpy(ran_nodes),edge_index=torch.from_numpy(edges.T).long())
  data.train_mask=torch.from_numpy(np.int32(np.random.random(size=data.x.shape[0])<0.8)).long()
  data.val_mask=1-data.train_mask
  data.y=torch.from_numpy(flabels).long()
  return data

data=create_data(X_training,y_training)
test_data=create_data(X_test,y_test)

IndexError: ignored

In [ ]:
np.array(test_data.y.cpu()).mean()

0.17315500358251731

In [ ]:

# Print information about the graph
print(f'\nGraph:')
print('------')
print(f'Training nodes: {sum(data.train_mask).item()}')
print(f'Evaluation nodes: {sum(data.val_mask).item()}')
#print(f'Test nodes: {sum(data.test_mask).item()}')
print(f'Edges are directed: {data.is_directed()}')
print(f'Graph has isolated nodes: {data.has_isolated_nodes()}')
print(f'Graph has loops: {data.has_self_loops()}')


#set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Create batches with neighbor sampling
#A NeighborLoader is a data loader that performs neighbor sampling for GNN's
#Allows for mini-batch training of GNNs on large-scale graphs where full-batch training is not feasible.
#num_neighbors denotes how many neighbors are sampled for each node in each iteration.
#https://pytorch-geometric.readthedocs.io/en/latest/modules/loader.html#torch_geometric.loader.NeighborLoader
train_loader = NeighborLoader(
    data.to(device),
    num_neighbors=[5, 5],
    batch_size=16,
)


testloader = NeighborLoader(
    test_data.to(device),
    num_neighbors=[5, 5],
    batch_size=16)

'''
GAT- uses Attention stratgey
compute the hidden representations of each node in the Graph by attending
over its neighbors using a self-attention strategy
'''
class GAT(torch.nn.Module):
    def __init__(self):
        super(GAT, self).__init__()
        self.hid = 8
        self.in_head = 8
        self.out_head = 1

        self.conv1 = GATConv(768, self.hid, heads=self.in_head, dropout=0.6)
        self.conv2 = GATConv(self.hid*self.in_head, 2, concat=False,
                             heads=self.out_head, dropout=0.6)
        self.optimizer = torch.optim.Adam(self.parameters(), lr=1e-4,weight_decay=5e-5)

    def forward(self,x, edge_index):
        # Dropout before the GAT layer is used to avoid overfitting
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index)
        return x


def accuracy(pred_y, y):
    """Calculate accuracy."""
    return ((pred_y == y).sum() / len(y)).item()

def train(gmodel, epochs):
    criterion = torch.nn.CrossEntropyLoss(weight=torch.tensor([1/0.82,1/0.18]).to(device))
    optimizer = gmodel.optimizer

    for epoch in range(epochs+1):
      total_loss = 0
      acc = 0
      val_loss = 0
      val_acc = 0
      gmodel.train()

      # Train on batches
      full_out=[]
      full_y=[]
      valout=[]
      valy=[]
      for batch in train_loader:
        optimizer.zero_grad()
        out = gmodel(batch.x.float(), batch.edge_index)
        loss = criterion(out[batch.train_mask], batch.y[batch.train_mask])
        total_loss += loss
        acc += accuracy(out[batch.train_mask].argmax(dim=1),
                        batch.y[batch.train_mask])
        full_out.append(np.array(out[batch.train_mask].argmax(dim=1).cpu()))
        full_y.append(np.array(batch.y[batch.train_mask].cpu()))
        loss.backward()
        optimizer.step()

        # Validation
        val_loss += criterion(out[batch.val_mask], batch.y[batch.val_mask])
        val_acc += accuracy(out[batch.val_mask].argmax(dim=1),
                            batch.y[batch.val_mask])

        valout.append(np.array(out[batch.val_mask].argmax(dim=1).cpu()))
        valy.append(np.array(batch.y[batch.val_mask].cpu()))

      valout=np.hstack(valout)
      valy=np.hstack(valy)
      full_out=np.hstack(full_out)
      full_y=np.hstack(full_y)
      print('mean',full_y.mean())
      print('omean',full_out.mean())

      # Print metrics every 10 epochs
      if(epoch % 2 == 0):
          f1 = sklearn.metrics.f1_score(full_y,
              full_out)
          vf1 = sklearn.metrics.f1_score(valy,
              valout)
          print(f'Epoch {epoch:>3} | Train Loss: {total_loss/len(train_loader):.3f} '
                f'Train F1: {f1:.3f}'
                f'| Train Acc: {acc/len(train_loader)*100:>6.2f}% | Val Loss: '
                f'{val_loss/len(train_loader):.2f} | Val Acc: '
                f'{val_acc/len(train_loader)*100:.2f}% '
                f'Val F1: {vf1:.3f}')
          test(gmodel)

def test(gmodel):

  total_loss = 0
  acc = 0
  criterion = torch.nn.CrossEntropyLoss(weight=torch.tensor([1/0.82,1/0.18]).to(device))

  # Train on batches
  full_out=[]
  full_y=[]
  gmodel.eval()
  for batch in testloader:
    with torch.no_grad():
      out = gmodel(batch.x.float(), batch.edge_index)
      loss = criterion(out, batch.y)
      total_loss += loss
      acc += accuracy(out.argmax(dim=1),batch.y)
      full_out.append(np.array(out.argmax(dim=1).cpu()))
      full_y.append(np.array(batch.y.cpu()))



  full_out=np.hstack(full_out)
  full_y=np.hstack(full_y)
  print(full_out.shape, full_y.shape,full_out.mean(),full_y.mean())

  f1 = sklearn.metrics.f1_score(full_y,
      full_out)
  print(f'Loss: {total_loss/len(testloader):.3f} '
        f'F1: {f1:.3f}'
        f'| Acc: {acc/len(testloader)*100:>6.2f}%')
  if f1>0.5:
    model.save_weights('babla.h5')

# Create GAT
gat = GAT().to(device)
print(gat)

# Train Graph Attention Network
train(gat, 8)



In [ ]:

test_set = ['ES2003', 'ES2004', 'ES2011', 'ES2014', 'IS1008', 'IS1009', 'TS3003', 'TS3004', 'TS3006', 'TS3007']
test_set = flatten([[m_id+s_id for s_id in 'abcd'] for m_id in test_set])
path_to_test=Path('test')

test_labels = {}
for transcription_id in test_set:
    G=create_dir_graph(transcription_id,path_to_test)

    fnodes=[]
    edges=[]
    for ind in G.nodes:
      node=G.nodes[ind]
      text=node['speaker']+': '+node['text']
      fnodes.append(text)
    for edge1,edge2 in G.edges:
      edges.append([edge1,edge2])

    encoded_nodes=encode_all(tokenizer,fnodes,256)

    ran_nodes=[]
    with torch.no_grad():
      for k in range(0,len(encoded_nodes['input_ids']),10):
        input_id=encoded_nodes['input_ids'][k:(k+10)]
        mask=encoded_nodes['attention_mask'][k:(k+10)]
        out = model.bert(input_ids=torch.from_numpy(input_id).to(device),attention_mask=torch.from_numpy(mask).to(device))
        ran_nodes.append(np.array(out.pooler_output.cpu()))
    ran_nodes=np.vstack(ran_nodes)
    edges=np.array(edges)
    data=torch_geometric.data.Data(x=torch.from_numpy(ran_nodes),edge_index=torch.from_numpy(edges.T).long()).to(device)
    data.y=np.arange(ran_nodes.shape[0])
    train_loader = NeighborLoader(data,
        num_neighbors=[5,10],
        batch_size=10)
    gat.eval()


    y_test=np.zeros(ran_nodes.shape[0])
    with torch.no_grad():
      for batch in train_loader:
        out = gat(batch.x, batch.edge_index)
        #out=model.classifier(batch.x)
        for key,val in zip(batch.y.cpu(),np.array(out.argmax(dim=1).cpu())):
          y_test[key]=val

    test_labels[transcription_id] = [int(a) for a in list(y_test)]

with open("test_labels_bert_baseline.json", "w") as file:
    json.dump(test_labels, file, indent=4)


/usr/local/lib/python3.10/dist-packages/torch_geometric/sampler/neighbor_sampler.py:60: UserWarning: Using 'NeighborSampler' without a 'pyg-lib' installation is deprecated and will be removed soon. Please install 'pyg-lib' for accelerated neighborhood sampling
  warnings.warn(f"Using '{self.__class__.__name__}' without a "


In [ ]:
!python make_submission.py --json_path test_labels_bert_baseline.json

In [ ]:
!mv "submission.csv" "drive/MyDrive/submission.csv"

In [ ]:
model.classifier

Linear(in_features=768, out_features=2, bias=True)